# 確認用

###  np.isscalarを試す

In [4]:
import numpy as np

print(np.isscalar(np.float64(1.0)))
print(np.isscalar(1.0))
print(np.isscalar(np.array(1.0)))
print(np.isscalar(np.array([1, 2, 3])))

True
True
False
False


### 可変長引数

In [1]:
def f(*x):
    print(x)

f(1)
f(1, 2, 3)
f(1, 2, 3, 4, 5, 6)

(1,)
(1, 2, 3)
(1, 2, 3, 4, 5, 6)
